In [2]:
#!/usr/bin/env python
# -*-coding:utf-8 -*-
"""
  ████
██    ██   Datature
  ██  ██   Powering Breakthrough AI
    ██
 
@File    :   rle_converter.ipynb
@Author  :   Leonard So
@Version :   1.0
@Contact :   hello@datature.io
@License :   Apache License 2.0
@Desc    :   COCO annotation file validity checker and additional utilities.
"""

'\n  ████\n██    ██   Datature\n  ██  ██   Powering Breakthrough AI\n    ██\n \n@File    :   rle_converter.ipynb\n@Author  :   Leonard So\n@Version :   1.0\n@Contact :   hello@datature.io\n@License :   Apache License 2.0\n@Desc    :   COCO annotation file validity checker and additional utilities.\n'

In [3]:
%pip install pycocotools
%pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from pycocotools import coco
import json
import cv2

Test the validity of your COCO annotation json file here, by entering the local path to your file below. If you encounter an error when loading, there is likely an issue with your COCO annotation file format.

json_file_path is the file path to your original COCO annotation file. output_file_path is the new annotation file path that will be created if there is a need for conversion.

In [5]:
json_file_path = "instances_default.json"
output_file_path = "new_annotation.json"

In [6]:
def coco_validity_checker(input_file_path: str):
    try:
        anno_file = coco.COCO(json_file_path)
        return anno_file
    except Exception as error:
        print("Your COCO file is not valid because of the following error: ", error)
        return None

In [9]:
anno_file = coco_validity_checker(json_file_path)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


# Converting RLE Encoded Segmentation to Polygon Annotations
Datature's upload format for segmentation masks is currently only for polygons. If needed, you can use this subsequent script to convert your RLE-encoded segmentation into polygon vertices. Typically, you know your segmentation is RLE-encoded if your annotation's segmentation component contains a dictionary with counts as a field.

Datature also currently doesn't support holes in segmentation masks. Please contact us if this is an issue for you!

In [10]:
def convert_rle_to_poly(anno_file: coco.COCO, output_file_path: str):
    new_annotation_list = []
    ann_id = 1
    for annotation in anno_file.anns.values():
        if type(annotation["segmentation"]) is list:
            print("These segmentation annotations should already be in polygon format! You don't need this.")
            return
        outputs = anno_file.annToMask(annotation)
        contours, _ = cv2.findContours(outputs, 
        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_TC89_KCOS)
        for cnt in contours:
            fixed_cnt = [cnt.flatten().tolist()]
            new_anno = annotation.copy()
            new_anno["segmentation"] = fixed_cnt
            new_anno["id"] = ann_id
            new_anno["iscrowd"] = 0
            ann_id += 1
            new_annotation_list.append(new_anno)

    with open(json_file_path) as f:
        orig_file = json.load(f)

    orig_file["annotations"] = new_annotation_list
    with open(output_file_path,"w") as out_f:
        json.dump(orig_file, out_f)

In [11]:
if anno_file is not None:
    convert_rle_to_poly(anno_file, output_file_path)
else:
    print("Please fix your annotation file to be a valid COCO file first!")